In [1]:
import openai
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
client = openai.Client()

In [2]:
import pandas as pd

dataset = pd.read_csv('full_netflix_dataset.csv')
dataset.head(3)

,url,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,https://www.netflix.com/title/60000724,Forrest Gump,Movie,"Drama, Romance",1994.0,tt0109830,8.8,2311997.0,"AD, AE, AG, AL, AO, AR, AT, AU, BA, BB, BE, BG..."
1,https://www.netflix.com/title/1154386,The Fifth Element,Movie,"Action, Adventure, Sci-Fi",1997.0,tt0119116,7.6,516291.0,"AT, CH, DE"
2,https://www.netflix.com/title/60031236,Kill Bill: Vol. 1,Movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1219973.0,"AE, AL, AO, AT, AU, AZ, BG, BH, BY, CA, CI, CM..."


### Cria o arquivo para ser enviado

In [6]:
file = client.files.create(
    file    = open('full_netflix_dataset.csv', 'rb'),
    purpose = 'assistants'
)
print(file)

FileObject(id='file-PsR3BFxCP9qbK9EuTNVE0lCc', bytes=3462237, created_at=1730267634, filename='full_netflix_dataset.csv', object='file', purpose='assistants', status='processed', status_details=None)


### Cria o [assistant](https://platform.openai.com/docs/assistants/overview/agents)
- `tool_resources` recebe os ids dos arquivos

In [8]:
DataAssistant = client.beta.assistants.create(
    name = "Engenheiro de Dados",
    instructions = 'Você é um Engenheiro de Dados que trabalha para a Netflix. Você recebeu o arquivo .CSV full_netflix_dataset.csv com informações importantes sobre o catálogo da Netflix. Sua tarefa é analisar o arquivo e responder as perguntas sobre os dados.',
    tools = [{'type': 'code_interpreter'}],
    tool_resources = {'code_interpreter': 
                        {'file_ids': 
                            [file.id]
                        }
                     },
    model = 'gpt-4o-mini'
)

### Cria e adiciona a mensagem em myThread

In [9]:
myThread = client.beta.threads.create()
print(myThread, end='\n\n')

message = client.beta.threads.messages.create( # cria uma mensagem
    thread_id = myThread.id, # na thread especificada
    role = 'user',
    content = 'Liste todos os elementos do catálogo da Netflix com maior nota imdb que estão disponíveis no Brasil e possuem mais de um milhão de votos. Ordene a lista resposta por nota imdb de forma decrescente e mostre qual foi.'
)
print(message)

Thread(id='thread_CsDR8JXgq2mN2vO4zMCvBPVy', created_at=1730268650, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

Message(id='msg_4EGKNFpN6j2rwhpLyNFQ003q', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Liste todos os elementos do catálogo da Netflix com maior nota imdb que estão disponíveis no Brasil e possuem mais de um milhão de votos. Ordene a lista resposta por nota imdb de forma decrescente e mostre qual foi.'), type='text')], created_at=1730268650, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_CsDR8JXgq2mN2vO4zMCvBPVy')


### Roda a thread no assistant

In [10]:
run = client.beta.threads.runs.create(
    thread_id    = myThread.id,
    assistant_id = DataAssistant.id,
    instructions = 'O nome do usuário é Davi Brilhante e ele está pensando em assinar o serviço de streaming da netflix.'
)
print(run)

Run(id='run_Y9Zt7tKZTgbB5LiHQROdMT98', assistant_id='asst_pAXuddZgVF2vAxDd7YnHoRvw', cancelled_at=None, completed_at=None, created_at=1730268819, expires_at=1730269419, failed_at=None, incomplete_details=None, instructions='O nome do usuário é Davi Brilhante e ele está pensando em assinar o serviço de streaming da netflix.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_CsDR8JXgq2mN2vO4zMCvBPVy', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


### Aguarda a run terminar
- [estados que uma run pode assumir](https://platform.openai.com/docs/assistants/deep-dive/run-lifecycle)

In [11]:
while run.status in ['queued', 'in_progress', 'cancelling']:
    run = client.beta.threads.runs.retrieve( # recupera os dados da run 
        thread_id = myThread.id, # na thread especificada
        run_id = run.id # especifica qual run recuperar os dados
    )
    print(run.status)


in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
completed


### Verifica a resposta

In [12]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
        thread_id = myThread.id
    )
    print(mensagens)
else:
    print('Erro:', run.status)

SyncCursorPage[Message](data=[Message(id='msg_FtJYsI2dLMBrxF2FbfCq1shX', assistant_id='asst_pAXuddZgVF2vAxDd7YnHoRvw', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Aqui estão os títulos disponíveis na Netflix Brasil que possuem uma nota IMDb superior a 0 e mais de um milhão de votos, ordenados por nota IMDb de forma decrescente:\n\n| Título                                   | Nota IMDb | Número de Votos |\n|------------------------------------------|-----------|------------------|\n| Breaking Bad                             | 9.5       | 2,222,701        |\n| The Godfather                            | 9.2       | 2,059,785        |\n| Pulp Fiction                             | 8.9       | 2,269,045        |\n| Forrest Gump                             | 8.8       | 2,311,997        |\n| Stranger Things                         | 8.7       | 1,385,625        |\n| Terminator 2: Judgment Day              | 8.6       | 1,199,152        |\n| Sa

### Acesso de mensagens

<details>
  <summary>------- Output Esperado -------</summary>

  - Breaking Bad
  - The Godfather
  - Pulp Fiction
  - Forrest Gump
  - Stranger Things
  - Terminator 2: Judgment Day
  - Saving Private Ryan
  - Whiplash
  - Django Unchained
  - Eternal Sunshine of the Spotless Mind
  - Shutter Island
  - The Walking Dead
  - The Hunger Games
</details>



In [15]:
for mensagem in mensagens.data[::-1]:
    role    = mensagem.role
    content = mensagem.content[0].text.value
    print(f'    {role}: {content}', end='\n\n')

    user: Liste todos os elementos do catálogo da Netflix com maior nota imdb que estão disponíveis no Brasil e possuem mais de um milhão de votos. Ordene a lista resposta por nota imdb de forma decrescente e mostre qual foi.

    assistant: Para realizar a solicitação, primeiro, vou examinar o conteúdo do arquivo enviado. Isso me permitirá entender como os dados estão estruturados e quais informações estão disponíveis. Após revisar os dados, poderei filtrar os elementos conforme solicitado. Vamos verificar o conteúdo do arquivo.

    assistant: O arquivo parece ser um CSV contendo informações sobre títulos disponíveis na Netflix. Aqui estão algumas colunas que o arquivo contém:

- `url`: URL do título na Netflix
- `title`: Título do filme ou série
- `type`: Tipo (filme ou série)
- `genres`: Gêneros
- `releaseYear`: Ano de lançamento
- `imdbId`: ID do IMDb
- `imdbAverageRating`: Nota média no IMDb
- `imdbNumVotes`: Número de votos no IMDb
- `availableCountries`: Países onde está dispon

### Analisando os passos do modelo

In [16]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id = myThread.id,
  run_id = run.id
)

In [19]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)


=== Step: message_creation

=== Step: tool_calls

=== Step: message_creation

=== Step: tool_calls

=== Step: message_creation

=== Step: tool_calls

=== Step: message_creation


In [18]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)
    
    if step.step_details.type == 'tool_calls':
            for tool_call in step.step_details.tool_calls:
                print('-----')
                print(tool_call.code_interpreter.input)
                print('-----')
                # print('Result')
                # print(tool_call.code_interpreter.outputs[0].logs)
                
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id  = myThread.id,
            message_id = step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)


=== Step: message_creation
Para realizar a solicitação, primeiro, vou examinar o conteúdo do arquivo enviado. Isso me permitirá entender como os dados estão estruturados e quais informações estão disponíveis. Após revisar os dados, poderei filtrar os elementos conforme solicitado. Vamos verificar o conteúdo do arquivo.

=== Step: tool_calls
-----
# Vamos ler o conteúdo do arquivo enviado para entender sua estrutura
file_path = '/mnt/data/file-PsR3BFxCP9qbK9EuTNVE0lCc'
with open(file_path, 'r') as file:
    content = file.readlines()

# A primeira linha geralmente contém os headers, vamos mostrar algumas linhas do arquivo
content[:10]  # Mostrando as primeiras 10 linhas para análise inicial.
-----

=== Step: message_creation
O arquivo parece ser um CSV contendo informações sobre títulos disponíveis na Netflix. Aqui estão algumas colunas que o arquivo contém:

- `url`: URL do título na Netflix
- `title`: Título do filme ou série
- `type`: Tipo (filme ou série)
- `genres`: Gêneros
- `rel